# Kubernetes

## 参考文献

* https://kubernetes.io/docs/home/
* *kubernetes up and running*
* *kubernetes in action*
* *kubernetes deep dive（深入剖析kubernetes）*

# 容器基础技术

![](https://bj.bcebos.com/ipic/k8s容器部署架构.png)

> 图片来自https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/

容器是一种沙盒技术。顾名思义，沙盒就是能够像集装箱一样把你的应用装起来。应用和应用之间因为有了边界而不至于互相干扰。

容器实际上是在创建容器的时候，指定了进程的一组Namaspace参数。这样，容器只能看到当前namespace所限制的资源、文件、状态或者配置。

**容器其实是一种特殊的进程而已。**

## 隔离和限制

Linxu容器是通过Namespace实现容器**隔离**。

Namespace技术实际上是修改了应用进程看待整个计算机的视图，只能看到指定的内容。

容器化的进程依然是宿主机上的普通进程，这就意味着不存在因为虚拟化产生的性能损耗。

Linux容器是通过Cgroups（Linux control groups）实现对资源的**限制**。Linux Cgroups最主要的作用就是限制进程能访问资源上限，例如CPU、内存、磁盘和网络带宽等。

## 深入理解容器镜像

Docker项目核心的原理实际上是为待创建的用户进程：

1. 启用Linux Namespace配置
2. 设置指定的Cgroups参数
3. 切换进程的根目录(change root)

**docker制作镜像时并没有沿用以前只做rootfs的流程，Docker在镜像设计中引入了层（layer）的概念，也就是说，用户制作镜像的每一步操作都会生成一个层，也就是一个增量的rootfs。** 这用到了一种叫做UnionFS(union file system 联合文件系统)的技术，最重要的能力时将不同位置的目录联合挂载到同一个目录下。

所谓的"镜像"，实际上是一个Ubuntu操作系统的rootfs，它的内容是Ubuntu操作系统的所有文件和目录。不过，不同之处在于，Docker镜像使用的是rootfs往往有多个层组成。

`docker inspect python:3.8`

```json
[
    ...
        "RootFS": {
            "Type": "layers",
            "Layers": [
                "sha256:afa3e488a0ee76983343f8aa759e4b7b898db65b715eb90abc81c181388374e3",
                "sha256:4b0edb23340c111e75557748161eed3ca159584871569ce7ec9b659e1db201b4",
                "sha256:5a9a65095453efb15a9b378a3c1e7699e8004744ecd6dd519bdbabd0ca9d2efc",
                "sha256:ad83f0aa5c0abe35b7711f226037a5557292d479c21635b3522e1e5a41e3ce23",
                "sha256:138b36d5a43a20d306eaf3797da44427d83f71eb55540035cfe79256137ba5d3",
                "sha256:515418ad7cad08309843c1321d57b9f07d0aa0c65d79e5b0c9c29fb88801f878",
                "sha256:3aaf323e88818ea7042177f429ae8d196ac7e514ffdcf0c3562641aed29b2054",
                "sha256:67d52d14a08fb66183fb1169a942a07c595c5a09b48f9a27e4bedd2764f6bed5",
                "sha256:9e8b2f58daa9ae15e266d1b2b031f22cbec6ca9bd03b8d574d24ec8cdc43cd8c"
            ]
    ...
]
```

`python:3.8`镜像由9个层组成，这个9个层就是增量的rootfs，每一层都是Ubuntu操作系统文件和目录的一部分。使用镜像时，Docker会把这些增量联合挂载一个统一的挂载点上。

## 重新认识Linux容器

```dockerfile
FROM python:3.8

# 设置环境变量
ENV NAME World

...

# 允许外接访问容器的80端口
EXPOSE 80

# 设置容器进程为：python app.py

CMD ["python" "app.py"]
```

**使用Dockerfile时，会看到"ENTRYPOINT"，他和CMDB都是Docker容器进程启动时的必须参数，完整的格式是：ENTRYPOINT CMD.**

**Docker默认提供了一个隐含的ENTRYPOINT，即`/bin/sh -c`。所以，在不指定ENTRYPOINT时，实际容器里运行的是`/bin/sh -c "python app.py"`，即CMD的内容就是ENTRYPOINT的参数。**

## 总结

实际上Docker容器是由Linux Namespace、Linux Cgroups和rootfs这3中技术构建出来的进程隔离环境。

一个正在运行的Linux容器，可以"一分为二"的看待：
1. 一组联合挂载在/var/lib/docker/aufs/mnt上的rootfs，这一部分成为**容器镜像**
2. 一个由Namespace+Cgroups构成的隔离环境，这一部分成为**容器运行时**

# Kubernetes设计和架构

![](https://bj.bcebos.com/ipic/k8s-architecture.png)

![](https://bj.bcebos.com/ipic/k8s项目的全局架构图.jpg)

Kubernetes项目最主要的设计思想是，以统一的方式抽象底层基础设施能力（比如计算、存储和网络），定义任务编排的各种关系，将这些抽象以声明式API的方式对外暴露，从而允许平台构建者基于这些抽象进一步构建自己的PaaS乃至任何上层平台。